####   Exemple

In [1]:
from Classes_Both.module_marche import DonneeMarche
from Classes_MonteCarlo_LSM.module_brownian import Brownian
from Classes_MonteCarlo_LSM.module_LSM import LSM_method
from Classes_Both.module_option import Option
from Classes_Both.derivatives import OptionDerivatives, OptionDerivativesParameters
import datetime as dt
import numpy as np
import time
from Classes_TrinomialTree.module_arbre_noeud import Arbre
import matplotlib.pyplot as plt
import pandas as pd

### Paramètre

In [2]:
start_date = dt.datetime(2025, 4, 4)
end_date = dt.datetime(2026, 4, 4)

market = DonneeMarche(date_debut= start_date,
volatilite=0.30, 
taux_interet=0.05, 
taux_actualisation=0.05,
# dividends=[{"ex_div_date": dt.datetime(2024, 4, 21), "amount": 3, "rate": 0}], 
dividende_ex_date = dt.datetime(2025, 1, 1),
dividende_montant = 0,
dividende_rate=0,
prix_spot=100)

optionA = Option(date_pricing=start_date, 
                maturite=end_date, 
                prix_exercice=100, call=False, americaine=True)

optionB = Option(date_pricing=start_date, 
                maturite=end_date, 
                prix_exercice=102, call=False, americaine=True)
# bermudien, 4 de chaque mois, donc mettre 1 pas de temps à chaque mois

optionC = Option(date_pricing=start_date, 
                maturite=end_date, # ici prix strike inultile, et type call put inutile
                prix_exercice=107, call=True, americaine=True)

pricer = LSM_method(optionC)

# nombre d'année entre début et fin 
period = (end_date - start_date).days / 365
nb_pas_arbre = 1000
arbre = Arbre(nb_pas_arbre, market, optionA, pruning = True)
arbre.pricer_arbre()

print(f"Prix arbre option {arbre.prix_option}")

Prix arbre option 9.867787904968338


### Test solo

In [3]:
nb_pas = 100 #12 #10000 #int((end_date - start_date).days / 2)*2
nb_chemin = 10000
seed = 1
method = 'vector'        # vector ou scalar
model_type="polynomial"  # "polynomial" "laguerre" "hermite" "legendre" "chebyshev"
degree_polynome = 3      # utile pour polynomial, laguerre, hermite, legendre, chebyshev

brownian = Brownian(period, nb_pas, nb_chemin, seed)
price, std_error, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=False,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price)
print("Ecart type: ", std_error)
print("Intervalle de confiance: ", intervalle)     


Nb chemins non antithetic: 10000
Prix min ('non antithetic', 'européenne', 'vector'): 9.283562945698417
Prix max ('non antithetic', 'européenne', 'vector'): 9.810675416196187
Prix:  9.547119180947302
Ecart type:  0.1317781176244422
Intervalle de confiance:  (np.float64(9.283562945698417), np.float64(9.810675416196187))


In [4]:
brownian = Brownian(period, nb_pas, nb_chemin, seed)
price_ant, std_error_ant, intervalle_ant = pricer.LSM(brownian, market, method=method, 
                              antithetic=True,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price_ant)
print("Ecart type: ", std_error_ant)
print("Intervalle de confiance: ", intervalle_ant)

#std_error_ant / std_error - 1 # reduction ecart type

Nb chemins antithetic: 20000
Prix min ('antithetic', 'européenne', 'vector'): 9.275807018096339
Prix max ('antithetic', 'européenne', 'vector'): 9.646759340761994
Prix:  9.461283179429167
Ecart type:  0.09273808066641355
Intervalle de confiance:  (np.float64(9.275807018096339), np.float64(9.646759340761994))


### Itération nombre pas et chemin

In [ ]:
###### test boucle
from itertools import product
liste_chemin = [10000 * i for i in range(1, 11)]
liste_pas_chemin = [(10000 * i, int((end_date - start_date).days / 2)) for i in range(1, 11)]
liste_pas = [10 * i for i in range(1, 31)]
combinations = list(product(liste_chemin,liste_pas))
dico_price = {}

def test_boucle(combi):
    for (path,pas) in combi:
        brownian = Brownian(period, pas, path, 1)
        price, std_error, intervalle = pricer.LSM(brownian, market, method='vector', antithetic=True, 
                                                  poly_degree=2, model_type="polynomial",
                                                  print_info = False)
        if intervalle[0] <= arbre.prix_option <= intervalle[1]:
            if int(arbre.prix_option * 100) / 100 == int(price * 100) / 100:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4), 'proche': "✅    "}
                break
            if int(price * 100) / 100 - 0.01 <= int(arbre.prix_option * 100) / 100 <= int(price * 100) / 100 + 0.01:# and std_error < 0.01:
                dico_price[(pas,path)] = {'vrai prix': arbre.prix_option,'price': round(price, 4) ,'ecart-type': round(std_error, 4), 
                'min': round(intervalle[0], 4), 'max':round(intervalle[1], 4),'proche':'❌    '}
        print(pas,path, price)

test_boucle(combinations)
test_boucle(liste_pas_chemin)
pd.DataFrame(dico_price)

10 10000 10.252561959718344
20 10000 10.24955386579374
30 10000 10.367117168656707
40 10000 10.576852400488375
50 10000 10.545147404644506
60 10000 10.572537461042511
70 10000 10.63403575603293
80 10000 10.698433115334879
90 10000 10.70061894970842
100 10000 10.745428859842105
110 10000 10.783574223732154
120 10000 10.789729203076806
130 10000 10.791381853999168
140 10000 10.861213609906988
150 10000 10.856888882653035
160 10000 10.819008194014334
170 10000 10.766966575531223
180 10000 10.827560877734955
190 10000 10.815424509224764
200 10000 10.82252016479841
210 10000 10.834142283567573
220 10000 10.859427239047422
230 10000 10.88377208619735
240 10000 10.846205000097983
250 10000 10.853774508496791


In [ ]:
nb_pas = int((end_date - start_date).days / 2)
nb_chemin = 10000
seed = 1
method = 'vector'        # vector ou scalar
model_type="polynomial"  # "polynomial" "laguerre" "hermite" "legendre" "chebyshev"
degree_polynome = 2      # utile pour polynomial, laguerre, hermite, legendre, chebyshev

# non antithetic
brownian = Brownian(period, nb_pas, nb_chemin, seed)
price, std_error_non_antithetic, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=False,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)

# antithetic
brownian = Brownian(period, nb_pas, nb_chemin, seed)
price_antithetic, std_error_antithetic, intervalle = pricer.LSM(brownian, market, method=method, 
                              antithetic=True,poly_degree=degree_polynome, model_type=model_type,
                              print_info = True)
print("Prix: ", price, std_error_non_antithetic)
print("Prix antithetic: ", price_antithetic, std_error_antithetic)
std_error_antithetic/std_error_non_antithetic - 1


Nb chemins non antithetic: 10000
Prix min ('non antithetic', 'américaine', 'vector'): 14.206988140219405
Prix max ('non antithetic', 'américaine', 'vector'): 15.024044644362519
Nb chemins antithetic: 10000
Prix min ('antithetic', 'américaine', 'vector'): 14.424295704137034
Prix max ('antithetic', 'américaine', 'vector'): 14.810330706710904
Prix:  14.615516392290962 0.20426412603577876
Prix antithetic:  14.617313205423969 0.09650875064346773


-0.5275296131707272

### Grecques

In [6]:
pricer_options = {
    "method": "vector",
    "poly_degree": 3,
    "model_type": "polynomial",
    "antithetic": False
}

brownian = Brownian(period, nb_pas, nb_chemin, seed)

option_deriv = OptionDerivatives(option, market, pricer, pricer_options)
 
#print("Prix :", option_deriv.price(option_deriv.parameters))
print("Delta :", option_deriv.delta(brownian))
print("Vega :", option_deriv.vega(brownian)/100)
print("Theta :", option_deriv.theta(brownian)/100)
print("Gamma :", option_deriv.gamma(brownian))
print("Vomma :", option_deriv.vomma(brownian)/100)


Delta : -0.37212091960992755
Vega : 0.38287310672401276
Theta : -0.03405000295189966
Gamma : 0.012178115046128646
Vomma : -0.018469466045356597


In [5]:
option_deriv.delta() * 20

np.float64(-8.7105062608671)